In [94]:
%matplotlib widget
%load_ext autoreload
%autoreload 2


import matplotlib.pyplot as plt
import matplotlib.gridspec as GridSpec
import numpy as np
import os
import time

from matplotlib.widgets import Button

# from tqdm import tqdm, tqdm_notebook
import math

from  scipy.special import digamma as ψ
from mpmath import besseli as Iν
from scipy.constants import hbar, pi, h, e, k

kB = k
γ = 0.57721566
RQ = h/4/e**2
Φ0 = h/2/e

from JJformulas import *
from JJ_data_processing import *

plt.close('all')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
Δ = 2.1*kB
R_env = 377

SyntaxError: invalid syntax (<ipython-input-142-0872fb3064c7>, line 1)

In [40]:
os.getcwd()

'e:\\OneDrive - Rutgers University\\files_GitHub\\JJ arrays'

In [4]:
EJ_AB(150e3)

0.04517241304820373

In [5]:
Ic (150e3)

1.8950499176670637e-09

# $E_J^* (E_J)$

In [51]:
data = np.genfromtxt(os.getcwd()+ f'\IcvsEJ.dat')
IcIAB_exp = data[:, 2]
EJ0_exp = data[:, 0]
EJ0_exp

array([8.13031, 0.65602, 0.36831, 0.1087 , 0.04361])

In [25]:
R_list = np.logspace(2, 6, 101)

R_env = 377
Tb = 0.05

# EJs = EJ_star (EJ = EJ_AB(R_list), R = II(R_env, Rj(R_list, Tb)), T = Tb, C = 1.95e-15)

EJs = EJ_star_simp (EJ = EJ_AB(R_list),  C = 1.95e-15)

fig, ax1 = plt.subplots()
ax1.plot( EJ_AB(R_list) , EJs, label = r'C = 1.95 fF')

ax1.plot( EJ_AB(R_list) , EJ_AB(R_list), '--k', alpha = 0.5)
# ax.plot( EJ_AB(R_list) , EJ_AB(R_list)**2 / 5, '--k', alpha = 0.5)

ax1.plot(EJ0_exp, EJ0_exp * IcIAB_exp, 'o' , label = 'exp. data')


ax1.legend()

ax1.set_xscale('log')
ax1.set_yscale('log')

ax1.set_xlabel(r'$E_J, K$')
ax1.set_ylabel(r'$E_J^*, K$')

FigureCanvasNbAgg()

Text(0, 0.5, '$E_J^*, K$')

# IVCs

In [52]:
Rs = [1e2, 10e3, 20e3, 150e3 ]

R_env = 2e5
Tb = 0.05
Cj = 1.5e-15

n = len(Rs)

fig = plt.figure(figsize = (7,10))
gs = GridSpec.GridSpec(n, 1)



Vs = np.linspace(0, 1e-2, 101)


# Rs = [ 200e3 ]

for i, R in enumerate(Rs):
    
    ax = fig.add_subplot(gs[i] )#,sharex=ax)

    Rsh = II(R_env, Rj(R, Tb))
    
    EJ_s = EJ_star_simp (EJ = EJ_AB(R),  C = Cj)
#     EJ_s = EJ_star (EJ = EJ_AB(R), R = Rsh, T = Tb, C = Cj)

    Is = I_IZ( Vs, EJ = EJ_s, R = Rsh, T = Tb) 


    ax.plot( (Vs - 1*Is*Rsh)/1e-3, (Is)/1e-9, '-', label = 'Rn = {} kOhm, EJ* = {:1.3f} K'.format(R/1e3, EJ_s))

    ax.legend()    
    ax.set_xlabel(r'$V, mV$')
    ax.set_ylabel(r'$I, nA$')

# ax.set_xscale('log')
# ax.set_yscale('log')

plt.tight_layout()

D:\anaconda\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if __name__ == '__main__':


FigureCanvasNbAgg()

# $R_0, I_{sw}(E_J)$


In [75]:
data = np.genfromtxt(os.getcwd()+ f'\IcvsEJ.dat')
IcIAB_exp = data[:, 2]
EJ0_exp = data[:, 0]
EJ0_exp

array([8.13031, 0.65602, 0.36831, 0.1087 , 0.04361])

In [76]:
data = np.genfromtxt(os.getcwd()+ f'\R0vsEJ.dat')
R0_exp = data[:, 3]
EJ0_exp = data[:, 2]
EJ0_exp/2

array([8.130315, 0.656025, 0.368315, 0.1087  , 0.04361 ])

In [77]:
fig, ax = plt.subplots()

ax.plot(R0_exp, IcIAB_exp, 'o-')
ax.set_xscale('log')
ax.set_yscale('log')

FigureCanvasNbAgg()

In [79]:
R_list = np.logspace(2, 5, 51)

R_env = 1e5


Cj = 1.5e-15
Tb = 0.05

# EJs = EJ_star_simp (EJ = EJ_AB(R_list), C = Cj)

fig, (ax1, ax2) = plt.subplots(2,1, sharex = True, figsize = (5,5))

R0s = []
Isw_maxs = []

for R in R_list:
    
    EJs = EJ_star_simp (EJ = EJ_AB(R), C = Cj)
#     EJs = EJ_star (EJ= EJ_AB(R), R = II(R_env, Rj(R, Tb)), T = Tb, C = Cj)
    R0, Isw_max = find_R0_Isw( EJ = EJs, R_env = R_env , T = Tb)    
    
    R0s.append(R0)
    Isw_maxs.append(Isw_max)

ax1.plot( EJ_AB(R_list) , R0s, label = r'C = {:1.2f} fF'.format(Cj/1e-15))

# ax1.plot( EJ_AB(R_list) , EJ_AB(R_list), '--k', alpha = 0.5)
# ax.plot( EJ_AB(R_list) , EJ_AB(R_list)**2 / 5, '--k', alpha = 0.5)

ax1.plot(EJ0_exp, R0_exp, 'o' , label = 'exp. data')


ax1.legend()

ax1.set_xscale('log')
ax1.set_yscale('log')

# ax1.set_xlabel(r'$E_J, K$')
ax1.set_ylabel(r'$R_0, k \Omega$')

ax2.plot( EJ_AB(R_list) , np.array(Isw_maxs) / Ic(R_list), label = r'C = {:1.1f} fF'.format(Cj/1e-15))
ax2.plot(EJ0_exp, IcIAB_exp, 'o' , label = 'exp. data')
ax2.legend()

ax2.set_xscale('log')
ax2.set_yscale('log')

ax2.set_xlabel(r'$E_J, K$')
ax2.set_ylabel(r'$I_{sw}/I_{AB}$')

plt.tight_layout()

D:\anaconda\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  # This is added back by InteractiveShellApp.init_path()


FigureCanvasNbAgg()

# $R_0(T)$

In [21]:
RN = 3e3
R_env = 200e3

Cj = 1.4e-15
Tb_list =  np.linspace(0.05, .85, 51)

EJs = EJ_star_simp (EJ = EJ_AB(RN), R = R_env, T = Tb, C = Cj)

fig, ax1 = plt.subplots()

R0s = []

for Tb in Tb_list:
    
    Rsh = II(R_env, Rj(RN, Tb))
    
    R0 = find_R0( RN, R_env = Rsh , T = Tb, C = Cj)
    R0s.append(R0)

ax1.plot( Tb_list , R0s, label = r'C = {:1.2f} fF'.format(Cj/1e-15))




ax1.legend()

# ax1.set_xscale('log')
ax1.set_yscale('log')

ax1.set_xlabel(r'$T, K$')
ax1.set_ylabel(r'$R, k \Omega$')

FigureCanvasNbAgg()

Text(0, 0.5, '$R, k \\Omega$')

# $I_{sw}, R_0 (R_{env})$ 

In [75]:
R_env_list = np.logspace(2, 6, 51)

RN = 7e3

Cj = 1.4e-15
Tb = 0.05


fig, ax1 = plt.subplots()

R0s = []
Isw_maxs = []

for R_env in R_env_list:

    EJs = EJ_star (EJ= EJ_AB(RN), R = R_env, T = Tb, C = Cj)
#     EJs = EJ_star_simp (EJ = EJ_AB(RN), C = Cj)
    
    R0, Isw_max = find_R0_Isw( EJ = EJs, R_env = R_env , T = Tb)
    
    R0s.append(R0)
    Isw_maxs.append(Isw_max)

ax1.plot( R_env_list , R0s, '-', label = r'$R_0$')

ax2 = ax1.twinx()
ax2.plot( R_env_list , np.array(Isw_maxs) / Ic(RN), '--', color = 'C1', label = r'$I_{sw}/I_c$')



ax1.set_xscale('log')
ax1.set_yscale('log')

ax1.set_xlabel(r'$R_{env}, Ohm$')
ax1.set_ylabel(r'$R_0, k \Omega$')

ax2.set_yscale('log')
ax2.set_ylabel(r'$I_{sw}/I_c$')

ax1.legend()
ax2.legend()

plt.tight_layout()

D:\anaconda\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if __name__ == '__main__':


FigureCanvasNbAgg()

# playing around

In [211]:
R_env = 1e7
Cj = 1.5e-15
# Tb = 0.3

EJ0= 3

fdir = 'E:\\OneDrive - Rutgers University\\files_py\\expdata\\'

Frust = 'IV scanB'
spl = 'chain 68N5\\' + Frust + '\\'

f = os.listdir(fdir + spl)[1]

IVC = load_IVC_B(file = fdir + spl+f )

fig = plt.figure(figsize = (9,9))
gs = GridSpec.GridSpec(4, 3)

for i in range(12):
# for i, j in enumerate([0,3,4,5,6,7,8, 11, 12, 14,15,16]):
    
    ax = fig.add_subplot(gs[i] )
    
    j = 1*i+5
    
    plot_IVC(ax, IVC[j])
    
#     plot_IVC(ax, IVC[j], cut = True)


    I, V = cut_dxdy(IVC[j]['I'], IVC[j]['V'], dx = 5e-8 ,dy = 2e-5)
    Isw = (np.max(I) - np.min(I))/2
#     ax.plot([-Isw, Isw], [0,0])
    
    B  = IVC[j]['B']
    Tb = IVC[j]['T']
    
    cosφ =  np.abs( np.cos(pi/2*B/8.85e-4))

    EJ = EJ0 * cosφ**1
#     EJ = EJ_star_simp (1.15* cosφ,  Cj)
    
    Ic0 = EJ/ (Φ0/2/pi/kB)
    Vc0 = R_env*Ic0
#     Vs = np.linspace(-Vc0*0.5, Vc0*0.5, 101)
    Vs = np.linspace(-1000e-4, 1000e-4, 101)
    
#     R0 = R0_IZ(EJ, R_env, Tb)
    R_sh = R_env #II(R0, R_env)
#     Tb = 1
    Is = I_IZ( Vs, EJ = EJ, R = R_sh, T = Tb) + 0*Iqp(Vs, Tb, V0 = 6e-4, G1 = 1/10e3,  G2 = 1/100e3)
#     ax.plot(Is,Vs - Is*R_sh, '-')

#     Rd = find_exp_R0(I,V)
#     ax.plot( V/ Rd ,V)
    
    print(Rd)
    
    plt.title('T = {:2.0f} mK'.format(Tb/1e-3))
    ax.legend()
    
    plt.tight_layout()

D:\anaconda\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  app.launch_new_instance()


FigureCanvasNbAgg()

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [299]:
fig, ax = plt.subplots()

j = 13

plot_IVC(ax, IVC[13], cut = True)
# plot_IVC(ax, IVC[13])

EJ = 0.0195 
R_env = 2.6e4


Vs = np.linspace(-4e-5, 4e-5)
Is = I_IZ( Vs, EJ = EJ, R = R_env, T = 0.05) + 0*Iqp(Vs, Tb, V0 = 6e-4, G1 = 1/10e3,  G2 = 1/100e3)
ax.plot(Is,Vs - Is*R_env - 2.0e-6, '-')

# Is = np.linspace(-0.5e-10, 0.5e-10)
# ax.plot(Is,1e-5*np.sinh(Is/0.35e-10), '-')

ax.legend()

print (EJ/IVC[13]['cosφ'])

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


FigureCanvasNbAgg()

0.08526585891339196


In [291]:
fig, ax = plt.subplots()

j = 5

# plot_IVC(ax, IVC[5])
plot_IVC(ax, IVC[j], cut = True)

EJ = 0.145
R_env = 3.09e4

Vs = np.linspace(-10e-5, 10e-5)
Is = I_IZ( Vs, EJ = EJ, R = R_env, T = 0.05) + 1*Iqp(Vs, Tb, V0 = 6e-4, G1 = 1/10e3,  G2 = 1/100e3)
ax.plot(Is - 1e-10 ,Vs - Is*R_env - 1.2e-6, '-')

ax.legend()

print (EJ/IVC[j]['cosφ'])

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


FigureCanvasNbAgg()

0.15272842348309862


In [276]:
fig, ax = plt.subplots()

j = 6

plot_IVC(ax, IVC[j], cut = True)
# plot_IVC(ax, IVC[13], cut = True)

EJ = 0.09
R_env = 1.39e4

Is = I_IZ( Vs, EJ = EJ, R = R_env, T = 0.05) + 1*Iqp(Vs, Tb, V0 = 6e-4, G1 = 1/10e3,  G2 = 1/100e3)
ax.plot(Is + 0e-10 ,Vs - Is*R_env + 0.5e-6, '-')

ax.legend()

print (EJ/IVC[j]['cosφ'])

FigureCanvasNbAgg()

0.10238572915318822


In [171]:
os.listdir(fdir + spl)

['68N5_scanB_050mK-030739.txt',
 '68N5_scanB_100mK-081113.txt',
 '68N5_scanB_300mK-114708.txt',
 '68N5_scanB_500mK-114708.txt']

In [210]:
R_env = 2e5
Cj = 1.5e-15
Tb = 0.3

j = 1

fdir = 'E:\\OneDrive - Rutgers University\\files_py\\expdata\\'

Frust = 'IV scanB'
spl = 'chain 68N5\\' + Frust + '\\'
# spl = 'chain 678N1\\' + Frust + '\\'

fig, ax = plt.subplots()

f = os.listdir(fdir + spl)[j]

ax.clear()

I, V = [], []

data = np.genfromtxt(fdir + spl+f, skip_header = 22 ) [1:,:] 
Ts    = data[:,5]
Iraw = data[:,7]
Vraw = data[:,8]
IG   = data[:,6]
    
# np.split(IG, 357)    
# _, n = np.unique(IG, return_counts = True) 

# np.split(IG, n) 

# Tb = np.mean(Ts)

index_sets = [np.argwhere(i == IG) for i in np.unique(IG)]

i = 0

Iss = []
Igs = []

R0s = []
R0_ths = []

for sll in index_sets[:]:
    sl = sll.flatten()

#     I, V = avg_group(Iraw[sl], Vraw[sl])
    I, V = Iraw[sl], Vraw[sl]
    I, V = cut_dxdy(I, V, dx = 1e-7 ,dy = 3.85e-4)
    
    Is = (np.max(I) - np.min(I))/2
    Ig = np.mean(IG[sl])
    
    Iss.append(Is)
    Igs.append(Ig)
  
    I, V = cut_dxdy(I, V, dx = 1e-9 ,dy = 3.85e-4)

    R0s.append( find_exp_R0(I, V) )
#     EJ = 3* np.abs( np.cos(pi/2*Ig/8.85e-4)    ) 
#     R0_th, _ = find_R0_Isw( EJ, R_env = 1e5 , T = 0.05, VERBOSE = False)
#     R0_ths.append(R0_th)


Iss = np.array(Iss)
Igs = np.array(Igs)

R0s = np.array(R0s)

cosφ =  np.abs( np.cos(pi/2*Igs/8.85e-4)    )    

Ic0 = Ic (2e3)

ax.plot(cosφ, Iss/Ic0/cosφ , 'o', label = r'exp data')

# ax.set_xscale('log')
ax.set_yscale('log')   

ax.set_xlabel(r'$cos(\pi \Phi / \Phi_0)$')
ax.set_ylabel(r'$I_{sw}/I_{c0}$')

ax2 = ax.twinx()
ax2.plot(cosφ, R0s , 'x', label = r'exp data')
ax2.set_yscale('log')
# Isw_maxs = []

# for cosφ_ in cosφ:
#     EJ0 = 0.25
#     EJs = EJ0*cosφ_**1
# #     EJs = EJ_star_simp (EJ = EJ_AB(RN), C = Cj)
    
#     R0, Isw_max = find_R0_Isw( EJ = EJs, R_env = R_env , T = Tb)
#     Isw_maxs.append(Isw_max/1e-9)
    
# ax.plot(cosφ, Isw_maxs, '-', label = 'IZ for EJ* = {:1.2f} K x cos(phi)'.format(EJ0))

ax.set_xlim( 0.1 ,1.2 )
ax.set_ylim( 1e-4,1e-1  ) 
 
# ax.legend()
EJ = EJ_AB(2e3)
# R0_KMs = R0_KM( EJ = EJ*cosφ, Ec = 0.9, Q = 5, T = 0.5)
R0_th = 2e3*np.exp( - (cosφ**1-1)*10.5) 
# R0_KMs = 2e3*np.exp( - (cosφ**2-1)*8) 


n, b = np.polyfit ( cosφ, np.log( R0s ), 1 )

print(n)

# R0_th = np.exp(n*cosφ+b)

ax2.plot(cosφ, R0_th , '--', label = r'th data')
# for c in cosφ:
    
#     R0_th  = R0_IZ(3.15*c, R = 1e6, T = 0.5)
#     R0_ths.append(R0_th)
# ax2.plot(cosφ, R0_ths , '--', label = r'th data')



D:\anaconda\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  del sys.path[0]


FigureCanvasNbAgg()

D:\anaconda\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
D:\anaconda\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan


In [188]:
np.polyfit 

<function numpy.polyfit(x, y, deg, rcond=None, full=False, w=None, cov=False)>

In [25]:
fig = plt.figure(figsize = (9,9))
gs = GridSpec.GridSpec(4, 3)

for i in range(12):
    ax = fig.add_subplot(gs[i] )
    plot_i(ax, 2*i, EJ0 = 1.2)
    plt.tight_layout()

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


FigureCanvasNbAgg()

D:\anaconda\lib\site-packages\numpy\lib\function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
D:\anaconda\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [402]:
Vs = np.linspace(-1e-2, 1e-2, 101)

fig, ax = plt.subplots()

for T in [ 0.1, 0.3, 0.5]:
    Is = Iqp(Vs, T, V0 = 1e-3, G1 = 1/1e3,  G2 = 1/30e3)

    ax.plot(Is, Vs)

FigureCanvasNbAgg()

In [81]:
fdir = 'E:\\OneDrive - Rutgers University\\files_py\\expdata\\'
Frust = 'IV ZF'
# Frust = 'IV scanB'
spl = 'chain 68N5\\' + Frust + '\\'
# spl = 'chain 678N1\\' + Frust + '\\'

fig, ax = plt.subplots()

def plot_i(ax, i):

    f = os.listdir(fdir + spl)[i]

    ax.clear()

    I, V = [], []
    
    data = np.genfromtxt(fdir + spl+f, skip_header = 22 ) [1:,:] 
    T = data[:,5]
    Iraw = data[:,7]
    Vraw = data[:,8] 

    ax.plot(Iraw, Vraw, 'o', alpha = 0.2, label = 'T = {:2.0f} mK'.format(np.mean(T*1e3)))

    ax.set_xlim(-4e-8, 4e-8)
    ax.set_ylim(-5e-4, 5e-4)
    
    ax.legend()
    fig.canvas.draw()
    time.sleep(.5)

def press( event, ax):

    global i


    if event.key == 'left':
        i -= 1 
        plot_i(ax, i)


    if event.key == 'right':
        i += 1 
        plot_i(ax, i)
        

    
i = 0
plot_i(ax, i)
fig.canvas.mpl_connect('key_press_event', lambda event: press(event, ax))
    

FigureCanvasNbAgg()

7

In [74]:
EC = 4*e**2/2/1e-15/kB

T = 0.2

E = np.linspace(-3, 3, 101)
f = np.exp (-E**2/ 4 /EC /T )

fig, ax = plt.subplots()

ax.plot(E,f)


D:\anaconda\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


FigureCanvasNbAgg()

In [44]:
find_R0_Isw( EJ = .01, R_env = 2e1 , T = 0.05)

(994.0098703697229, 1.4322014399176624e-11)

In [139]:
class jj( ):
    def __init__(self, **kwargs ):
        self.vars = ['Ic', 'RN', 'Rj', 'EJ', 'T', 'C', 'EC']
        
        self.Δ = 2.1*kB
        
        for var in self.vars:
            if var not in kwargs.keys():
                kwargs[var] = np.nan
                
            setattr(self,'_' + var, kwargs[var])        
        

        
    def Ic(self):
        if self._Ic is np.nan:
            self._Ic = self.EJ()/ (Φ0/2/pi/kB)
        return self._Ic

    def EJ(self):
        if self._EJ is np.nan:
            self._EJ = self._Ic* (Φ0/2/pi/kB)
        return self._EJ
    
    def RN(self):
        if self._RN is np.nan:
            self._RN = self.Δ / 4 /e /kB /self.EJ()
        return self._RN
    
#     def RN(self):
#         self._RN = self.Δ / 4 /e /kB /self.EJ()
#         return self._RN 

In [140]:
RN_list = np.array([1e3,2e3])
Ic_list = np.array([1e-9,2e-9])


jj(Ic = Ic_list).RN()

array([1.37466311e+20, 6.87331553e+19])

In [70]:
fdir = 'E:\\OneDrive - Rutgers University\\files_py\\expdata\\'

Frust = 'IV scanB'
spl = 'chain 68N5\\' + Frust + '\\'
f = os.listdir(fdir + spl)[0]


data = np.genfromtxt( fdir + spl+f, skip_header = 22 ) [1:,:] 
Ts    = data[:,5]
Ts

array([0.0500296, 0.0500648, 0.050029 , ..., 0.0499659, 0.0499395,
       0.0499333])

## KAutz

In [95]:
R_env = 1.49e4
Cj = 1.5e-15
# Tb = 0.3

EJ0= 1

fdir = 'E:\\OneDrive - Rutgers University\\files_py\\expdata\\'

Frust = 'IV scanB'
spl = 'chain 68N5\\' + Frust + '\\'

f = os.listdir(fdir + spl)[0]

IVC = load_IVC_B(file = fdir + spl+f )

fig = plt.figure(figsize = (9,9))
gs = GridSpec.GridSpec(4, 3)

for i, j in enumerate([0,3,4,5,6,7,8, 11, 12, 14, 15, 16]):
    
    ax = fig.add_subplot(gs[i] )
    
#     j = 2*i+0
    
#     plot_IVC(ax, IVC[j])
    
    plot_IVC(ax, IVC[j], cut = True)


    I, V = cut_dxdy(IVC[j]['I'], IVC[j]['V'], dx = 5e-9 ,dy = 2e-5)
    Isw = (np.max(I) - np.min(I))/2
#     ax.plot([-Isw, Isw], [0,0])
    
    B  = IVC[j]['B']
    Tb = IVC[j]['T']
    
    cosφ =  np.abs( np.cos(pi/2*B/8.85e-4))

    EJ = EJ0 * cosφ**1
#     EJ = EJ_star_simp (1.15* cosφ,  Cj)
    Q = 16
#     Q = Qp(EJ = EJ, Ec = 0.9, Rsh = 4e4)

    Ic0 = EJ/(Φ0/2/pi/kB)
    Vc0 = R_env*Ic0

    Is = np.linspace(-Ic0/Q, Ic0/Q, 101)
    
    R_sh = R_env #II(R0, R_env)
    Vs = V_KM(Is/Ic0, EJ = EJ, Ec = 0.9, Q = Q, T = .7)
    ax.plot(Is,Vs , '-')

    
    print(R0_KM( EJ = EJ, Ec = 0.9, Q = Q, T = .7))
    
    plt.title('T = {:2.0f} mK'.format(Tb/1e-3))
    ax.legend()
    
    plt.tight_layout()

FigureCanvasNbAgg()

14260.15637007266
14656.670807150254
15331.941477535633
16387.8482969361
19986.224483553357
26641.908396471183
38989.065495538416
139797.88651595337
169842.935413269
266393.1147719066
349802.85125544743
487263.40032403526


In [217]:
np.allclose(1e-8,2e-8)

True